# Домашнее задание к лекции "Базовые понятия статистики"

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

## Пункт 1 - знакомство с данными


In [1]:
import pandas as pd
import numpy as np

In [2]:
# для наглядности вынесем имена столбцов из horse_names, чтобы применить их к датасету
names = ['surgery', 'age', 'hospital_number', 'rectal_temp', 'pulse', 'resp_rate', 'temp_of_extremities',
            'peripheral_pulse', 'mucous_membranes', 'capillary_refill_time', 'pain', 'peristalsis', 'abdominal_distension',
            'nasogastric_tube', 'nasogastric_reflux','nasogastric_reflux_PH', 'rectal_examination', 'abdomen',
            'packed_cell_volume', 'total_protein', 'abdomin_appearance', 'abdom_total_protein', 'outcome',
            'surgical_lesion', 'site_of_lesion', 'type_of_lesion', 'subtype_of_lesion', 'cp_data']

In [3]:
len(names)

28

In [4]:
horse_df = pd.read_csv('horse_data.csv', na_values=['?'], header=None, names=names)
horse_df.head()

,surgery,age,hospital_number,rectal_temp,pulse,resp_rate,temp_of_extremities,peripheral_pulse,mucous_membranes,capillary_refill_time,...,packed_cell_volume,total_protein,abdomin_appearance,abdom_total_protein,outcome,surgical_lesion,site_of_lesion,type_of_lesion,subtype_of_lesion,cp_data
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0,NaN,2.0,...,45.0,8.4,NaN,NaN,2.0,2,11300,0,0,2
1,1.0,1,534817,39.2,88.0,20.0,NaN,NaN,4.0,1.0,...,50.0,85.0,2.0,2.0,3.0,2,2208,0,0,2
2,2.0,1,530334,38.3,40.0,24.0,1.0,1.0,3.0,1.0,...,33.0,6.7,NaN,NaN,1.0,2,0,0,0,1
3,1.0,9,5290409,39.1,164.0,84.0,4.0,1.0,6.0,2.0,...,48.0,7.2,3.0,5.3,2.0,1,2208,0,0,1
4,2.0,1,530255,37.3,104.0,35.0,NaN,NaN,6.0,2.0,...,74.0,7.4,NaN,NaN,2.0,2,4300,0,0,2


In [5]:
horse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                299 non-null    float64
 1   age                    300 non-null    int64  
 2   hospital_number        300 non-null    int64  
 3   rectal_temp            240 non-null    float64
 4   pulse                  276 non-null    float64
 5   resp_rate              242 non-null    float64
 6   temp_of_extremities    244 non-null    float64
 7   peripheral_pulse       231 non-null    float64
 8   mucous_membranes       253 non-null    float64
 9   capillary_refill_time  268 non-null    float64
 10  pain                   245 non-null    float64
 11  peristalsis            256 non-null    float64
 12  abdominal_distension   244 non-null    float64
 13  nasogastric_tube       196 non-null    float64
 14  nasogastric_reflux     194 non-null    float64
 15  nasoga

In [6]:
# Проверим, есть ли полные дубликаты строк
horse_df.drop_duplicates().shape
# Количество строк осталось неизменным, дубликатов не найдено

(300, 28)

Для дальнейшей работы выберу следующие столбцы:
- возраст (age) - категориальные данные
- ректальная температура (rectal_temp) - непрерывные данные (в градусах Цельсия)
- пульс (pulse) - дискретные данные (количество ударов в минуту)
- температура конечностей (temp_of_extremities) - категориальные данные
- перистальтика (peristalsis) - категориальные данные
- вздутие живота (abdominal_distension) - категориальные данные
- гематокрит (packed_cell_volume) - непрерывные данные (объем крови, приходящийся на эритроциты в %)
- общий уровень белка (total_protein) - непрерывные данные (gms/dL)


In [7]:
horse_filtered = horse_df.loc[:, ['age', 'rectal_temp', 'pulse', 'temp_of_extremities',
                                  'peristalsis', 'abdominal_distension','packed_cell_volume', 'total_protein']]
horse_filtered.head()

,age,rectal_temp,pulse,temp_of_extremities,peristalsis,abdominal_distension,packed_cell_volume,total_protein
0,1,38.5,66.0,3.0,4.0,4.0,45.0,8.4
1,1,39.2,88.0,NaN,4.0,2.0,50.0,85.0
2,1,38.3,40.0,1.0,3.0,1.0,33.0,6.7
3,9,39.1,164.0,4.0,4.0,4.0,48.0,7.2
4,1,37.3,104.0,NaN,NaN,NaN,74.0,7.4


In [8]:
# получим описание базовых метрик
horse_filtered.describe()

,age,rectal_temp,pulse,temp_of_extremities,peristalsis,abdominal_distension,packed_cell_volume,total_protein
count,300.000000,240.000000,276.000000,244.000000,256.000000,244.000000,271.000000,267.000000
mean,1.640000,38.167917,71.913043,2.348361,2.917969,2.266393,46.295203,24.456929
std,2.173972,0.732289,28.630557,1.045054,0.976744,1.065131,10.419335,27.475009
min,1.000000,35.400000,30.000000,1.000000,1.000000,1.000000,23.000000,3.300000
25%,1.000000,37.800000,48.000000,1.000000,3.000000,1.000000,38.000000,6.500000
50%,1.000000,38.200000,64.000000,3.000000,3.000000,2.000000,45.000000,7.500000
75%,1.000000,38.500000,88.000000,3.000000,4.000000,3.000000,52.000000,57.000000
max,9.000000,40.800000,184.000000,4.000000,4.000000,4.000000,75.000000,89.000000


### столбец age
Видим странное: согласно описанию из файла этот столбец принимает значения либо 1 (взрослая лошадь), либо 2 (молодая лошадь), однако здесь мы имеем максимальное значение 9. 

In [9]:
horse_filtered.age.unique().tolist()

[1, 9]

Уникальных значений этого столбца получили всего два, как и должно быть. Можно предположить, что вследствии неизвестной ошибки все значения "2" были заменены на "9". Либо столбец "age" мог быть перепутан с соседним столбцом "surgery" исходного датасета, который так же принимает значения только 1 и 2, и какая-то ошибка с заменой 2 на 9 произошла в "surgery"

Для этой задачи я предположу, что столбец "age" разделен на группы "взрослые лошади" - "молодые лошади" правильно (по признаку 1 и 9 соответственно) и, чтобы соответствовать описанию исходных данных заменю все 9 на 2

92% всех данных относится к взрослым лошадям, 8% к молодым

In [10]:
horse_filtered.loc[horse_filtered['age']==9, 'age'] = 2
horse_filtered.age.value_counts(normalize=True)

1    0.92
2    0.08
Name: age, dtype: float64

### Столбец rectal_temp

Есть пропуски в данных.

Нормальная температура составляет 37,8 С, может иметь незначительные суточные колебания.

25% квантиль соответствует нормальной температуре, т.е. у большинства лошадей температура повышенная.

Среднее значение так же соответствует повышенной температуре. 

Диапазон значений от 35.4 С до 40.8 С. Среднеквадратическое отклонение сравнительно невелико, разброс относительно среднего невелик, скорее всего выбросов здесь мы не увидим 

### Столбец pulse

Есть пропуски в данных.

Нормальный пульс взрослой лошади - 30-40 ударов в минуту.

25% квантиль соответствует 48 ударам в минуту, т.е. у большинства лошадей пульс так же повышен.

Диапазон значений здесь широкий: от 30 до 184 ударов в минуту, std так же довольно велико (28.63). Вероятно, здесь мы можем увидеть выбросы сверху. 

### Столбец temp_of_extremities
Есть пропуски в данных.

Нормальному состоянию животного соответствует значение 1

Для значения 2 (теплые), возможно, надо будет проверять взаимосвязь с показателем температуры, горячие конечности коррелируют с повышенной температурой

Мода для этого признака = 3 (прохладные конечности). Прохладные конечности указывают на возможный шок животного, что должно соотносится с высоким пульсом


In [12]:
horse_filtered.temp_of_extremities.mode()

0    3.0
dtype: float64

### Столбец peristalsis
Есть пропуски в данных

Нормальному состоянию животного соответствует значение 2

Мода для этого признака = 3 (пониженная активность кишечника). Норме соответствует только ~6% данных


In [27]:
print('mode', horse_filtered.peristalsis.mode(), sep='\n')
print('values', horse_filtered.peristalsis.value_counts(normalize=True),sep='\n')

mode
0    3.0
dtype: float64
values
3.0    0.500000
4.0    0.285156
1.0    0.152344
2.0    0.062500
Name: peristalsis, dtype: float64


### Столбец abdominal_distension
Есть пропуски в данных

Нормальному состоянию животного соответствует значение 1

Мода для этого признака = 1 (нет вздутия). Но если мы посмотрим процентное соотношение всех значений, увидим, что порядка 69% данных относятся к признакам 2-4, указывающие на наличие вздутия живота, которое может требовать операционного вмешательства.

Вздутие живота соотносится с пониженной перистальтикой

In [18]:
print('mode', horse_filtered.abdominal_distension.mode(), sep='\n')
print('values', horse_filtered.abdominal_distension.value_counts(normalize=True),sep='\n')

mode
0    1.0
dtype: float64
values
1.0    0.311475
3.0    0.266393
2.0    0.266393
4.0    0.155738
Name: abdominal_distension, dtype: float64


### Столбец packed_cell_volume
Есть пропуски в данных

нормальный диапазон от 30 до 50%. Уровень повышается по мере нарушения кровообращения или по мере обезвоживания животного.

75% квантиль соответствует 52%, т.е. у большинства животных доля эритроцитов в крови близка к нормальной или ниже нормы

Диапазон значений от 23.0% до 75.0%, std = 10.41 - ожидаем увидеть существенный разброс значений относительно среднего, возможно, будут выбросы.

###  Столбец total_protein
Есть пропуски в данных

нормальные значения лежат в диапазоне 6-7,5 г/дЛ

Диапазон между 25% квантилью и медианой соответствует нормальным показателям, однако на уровне 75% квантили видим резкое увеличение показателя буквально на порядок (57 г/дЛ). Этот столбец стоит внимательно исследовать на выбросы. Действительно ли возможно такое увеличение белка в случае болезни или имела место, например, ошибка связанная с неверными единицами измерения? Диапазон белка может быть измерен в граммах на децилитр как в нашем примере, а может в граммах на литр,
как например здесь: https://www.rossdales.com/laboratories/tests-and-diseases/proteins-total-protein-albumin-globulin
если интерпретировать данные, которые попали в диапазон от 75% квантили до максимума, как записанные в единицах г/Л и перевести их в г/дЛ, то получим данные, колеблющиеся около нормальных значений.

## Пункт 2 - работа с выбросами


In [12]:
def find_outliers_in_column(column):
    q25 = column.quantile(0.25)
    q75 = column.quantile(0.75)
    iqr = q75 - q25
    
    lower_outliers = column.loc[lambda x: x < q25 - (1.5 * iqr)].tolist()
    higher_outliers = column.loc[lambda x: x > q75 + (1.5 * iqr)].tolist()
    
    if lower_outliers or higher_outliers:
        print(f'В столбце {column.name} выбросов снизу - {len(lower_outliers)}, выбросов сверху - {len(higher_outliers)}')
    return lower_outliers, higher_outliers


In [101]:
# посчитаем по общей формуле выбросы для всех столбцов

for col in horse_filtered.columns:
    find_outliers_in_column(horse_filtered[col])


В столбце age выбросов снизу - 0, выбросов сверху - 24
В столбце rectal_temp выбросов снизу - 7, выбросов сверху - 7
В столбце pulse выбросов снизу - 0, выбросов сверху - 5
В столбце peristalsis выбросов снизу - 39, выбросов сверху - 0
В столбце packed_cell_volume выбросов снизу - 0, выбросов сверху - 3


Для **столбца age** 24 выброса сверху - это интерпретация программой значений 2 (молодые лошади). Говорить о них как о выбросах некорректно, к тому же это категориальные данные. Здесь оставим все без изменений

В **столбце rectal_temp** изначально я не ожидала увидеть выбросов, однако они найдены и сверху, и снизу.
Посмотрим на них:


In [13]:
lower_outliers, higher_outliers = find_outliers_in_column(horse_filtered['rectal_temp'])
horse_filtered[horse_filtered['rectal_temp'].isin(lower_outliers)]

В столбце rectal_temp выбросов снизу - 7, выбросов сверху - 7


,age,rectal_temp,pulse,temp_of_extremities,peristalsis,abdominal_distension,packed_cell_volume,total_protein
44,1,35.4,140.0,3.0,4.0,NaN,57.0,69.0
80,1,36.4,98.0,3.0,3.0,2.0,47.0,6.4
118,1,36.5,78.0,1.0,3.0,1.0,34.0,75.0
141,1,36.0,42.0,NaN,NaN,NaN,64.0,6.8
238,1,36.1,88.0,3.0,3.0,2.0,45.0,7.0
251,1,36.6,42.0,3.0,4.0,1.0,52.0,7.1
298,1,36.5,100.0,3.0,3.0,3.0,50.0,6.0


Низкая тепература тела соотносится с другими показателями - температура конечностей холодная, перистальтика гипомоторная или отсутствует, пульс частый, слабый. Резкое понижение температуры характерно для упадка сердечной деятельности (https://veterinarua.ru/svinovodstvo/103-klinicheskaya-diagnostika-vnutrennikh-boleznej-domashnikh-zhivotnykh-a-v-sinjov-1946-moskva-1946/1954-gipotermiya.html), а сердечная недостаточность м.б. одним из осложнений заболеваний ЖКТ, вызывающих колики.  Это согласуется с  указанием из файла описания данных: температура после повышения в начале болезни, может снова упасть, когда у лошади наступает состояние шока. Буду считать, что эти выбросы - не ошибки или искажения, а реальные данные, соответствующие симптомам болезни. Удалять или изменить их в датасете я не буду

In [14]:
horse_filtered[horse_filtered['rectal_temp'].isin(higher_outliers)]

,age,rectal_temp,pulse,temp_of_extremities,peristalsis,abdominal_distension,packed_cell_volume,total_protein
20,1,39.9,72.0,1.0,4.0,4.0,46.0,6.1
54,1,40.3,114.0,3.0,3.0,3.0,57.0,8.1
75,2,39.7,100.0,3.0,3.0,NaN,48.0,57.0
91,1,40.3,114.0,3.0,3.0,3.0,57.0,8.1
99,1,39.6,108.0,3.0,4.0,3.0,59.0,8.0
259,1,40.8,72.0,3.0,3.0,1.0,54.0,7.4
281,1,40.0,78.0,3.0,3.0,1.0,66.0,6.5


Повышенная температура так же не демонстрирует каких-либо совершенно невозможных цифр. В случае с молодой лошадью (строка 75, age = 2) это выглядит, как повышенная температура, но не выброс, так как у молодняка в целом средние значения температуры выше, чем у взролых лошадей (https://www.equinevet.ru/post/%D1%84%D0%B8%D0%B7%D0%B8%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5-%D0%BF%D0%BE%D0%BA%D0%B0%D0%B7%D0%B0%D1%82%D0%B5%D0%BB%D0%B8-%D0%BB%D0%BE%D1%88%D0%B0%D0%B4%D0%B8). В остальных случаях такая температура может указывать на наличие инфекционных заболеваний (может достигать 40-41 С). 

Однако выглядит неочевидным факт, что у лошадей с явно высокой температурой столбец temp_of_extremities показывает, что конечности нормальной температуры или прохладные. Из файла описания данных мы знаем, температура конечностей должна коррелировать с ректальной температурой, теплые конечности при повышенной температуре. Это же согласуется с бытовыми интуитивными представлениями. Поэтому для этих строк изменять значения температур я не буду, но в столбце temp_of_extremities укажу значения 2 (теплые конечности)

In [15]:
horse_filtered.loc[horse_filtered['rectal_temp'].isin(higher_outliers), 'temp_of_extremities'] = 2
horse_filtered[horse_filtered['rectal_temp'].isin(higher_outliers)]

,age,rectal_temp,pulse,temp_of_extremities,peristalsis,abdominal_distension,packed_cell_volume,total_protein
20,1,39.9,72.0,2.0,4.0,4.0,46.0,6.1
54,1,40.3,114.0,2.0,3.0,3.0,57.0,8.1
75,2,39.7,100.0,2.0,3.0,NaN,48.0,57.0
91,1,40.3,114.0,2.0,3.0,3.0,57.0,8.1
99,1,39.6,108.0,2.0,4.0,3.0,59.0,8.0
259,1,40.8,72.0,2.0,3.0,1.0,54.0,7.4
281,1,40.0,78.0,2.0,3.0,1.0,66.0,6.5


В **столбце pulse**, как и предполагалось, есть несколько выбросов сверху. Посмотрим на них

In [104]:
lower_outliers, higher_outliers = find_outliers_in_column(horse_filtered['pulse'])
horse_filtered[horse_filtered['pulse'].isin(higher_outliers)]

В столбце pulse выбросов снизу - 0, выбросов сверху - 5


,age,rectal_temp,pulse,temp_of_extremities,peristalsis,abdominal_distension,packed_cell_volume,total_protein
3,2,39.1,164.0,4.0,4.0,4.0,48.0,7.2
41,2,39.0,150.0,NaN,NaN,NaN,47.0,8.5
55,2,38.6,160.0,3.0,3.0,4.0,38.0,NaN
255,2,38.8,184.0,1.0,1.0,3.0,33.0,3.3
275,2,38.8,150.0,1.0,3.0,2.0,50.0,6.2


Заметим, что все случаи относятся к лошадям молодого возраста (age=2). Возраст может быть причиной повышенного пульса: если для взрослой лошади норма ~40 ударов в минуту, то для жеребят норма до 120 ударов, а у новорожденных до 150 ударов в минуту (https://www.equinevet.ru/post/%D1%84%D0%B8%D0%B7%D0%B8%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5-%D0%BF%D0%BE%D0%BA%D0%B0%D0%B7%D0%B0%D1%82%D0%B5%D0%BB%D0%B8-%D0%BB%D0%BE%D1%88%D0%B0%D0%B4%D0%B8).

Повышенна температура, гипомоторная перистальтика, вздутие живота, повышенный уровень белка (может наблюдаться при острых и хронических инфекционных заболеваниях, при обезвоживании) или наоборот пониженный уровнем белка (при нарушениях функциях ЖКТ) - признаки угнетенного организма. В этих случаях тахикардия возможна. Буду считать эти выбросы не ошибочными, а реальными данными, изменять их не буду

**Столбец peristalsis** содержит категориальные данные, анализировать его с точки зрения выбросов не буду

**Столбец packed_cell_volume** содержит выбросы сверху. Посмотрим на них

In [105]:
lower_outliers, higher_outliers = find_outliers_in_column(horse_filtered['packed_cell_volume'])
horse_filtered[horse_filtered['packed_cell_volume'].isin(higher_outliers)]

В столбце packed_cell_volume выбросов снизу - 0, выбросов сверху - 3


,age,rectal_temp,pulse,temp_of_extremities,peristalsis,abdominal_distension,packed_cell_volume,total_protein
4,1,37.3,104.0,NaN,NaN,NaN,74.0,7.4
144,1,37.1,84.0,3.0,4.0,4.0,75.0,81.0
151,1,39.3,NaN,4.0,4.0,2.0,75.0,NaN


Превышение нормы pcv составляет порядка 25%. Однако можно найти статьи (https://iopscience.iop.org/article/10.1088/1742-6596/790/1/012035/pdf), в которых упоминается, что такие показатели так же возможны, хоть и специфичны (в статье используется термин HCT вместо PCV, но это синонимы https://helix.ru/kb/item/42/). Высокий показатель pcv говорит о вязкости и густоте крови, это одно из последствий обезвоживания. Обезвоживание в свою очередь встречается при заболеваниях ЖКТ, вызывающих колики. Удалять или изменять эти данные pcv я не буду.  

Интересно, что в **столбце total_protein** не выявлено выбросов, хотя максимальные значения превышают нормальные значения более, чем в 10 раз. Мне не удалось найти источники, в которых указано, что такие показатели возможны или наоборот невозможны. 

In [106]:
# при норме в 6 - 7.5 г/дЛ посмотрим сколько записей с общим уровнем белка, превышающим, допустим, вдвое верхнюю границу нормы
horse_filtered[horse_filtered['total_protein'] > 15]

,age,rectal_temp,pulse,temp_of_extremities,peristalsis,abdominal_distension,packed_cell_volume,total_protein
1,1,39.2,88.0,NaN,4.0,2.0,50.0,85.0
14,1,38.2,76.0,3.0,4.0,1.0,46.0,81.0
23,2,38.3,130.0,NaN,4.0,NaN,50.0,70.0
24,1,38.1,60.0,3.0,4.0,3.0,51.0,65.0
29,1,37.7,48.0,2.0,1.0,1.0,45.0,76.0
...,...,...,...,...,...,...,...,...
290,1,38.6,45.0,2.0,1.0,NaN,43.0,58.0
293,1,NaN,78.0,3.0,3.0,NaN,43.0,62.0
294,1,38.5,40.0,1.0,1.0,1.0,37.0,67.0
295,1,NaN,120.0,4.0,4.0,NaN,55.0,65.0


В идеале требуется консультация ветеринара. В отсутствие специалиста, можно только предполагать. Если бы таких записей было сравнительно мало, логично было бы предположить ошибку измерений, ввода данных или использование единицы измерения граммы на литр вместо граммов на децилитр (в этом случае можно было бы исправить данные, поделив их на 10). Но так как таких строк болше 25% от общего числа, достаточно много, возможно это говорит о том, что данные не ошибочны, а просто выходят за пределы интуитивного восприятия человека, не сведущего в ветеринарии. Изменять их так же не буду.

## Пункт 3 - работа с пропущенными значениями

Так как в датасете немного записей (300 строк), цена удаления строк при очистке относительно велика. Постараюсь избегать ситуаций, когда необходимо удалять крупные группы строк. Вместо этого попробую варианты с заменой значений.

Единственное, перед работой с каждым столбцом в отдельности, проверю датасет на строки, в которых много пропусков данных. 

In [18]:
# Предположим, что для корректной работы нам нужн строки, в которых хотя бы половина столбцов заполнена
horse_filtered.dropna(thresh=4, inplace=True)
horse_filtered.info()
# Датасет стал меньше на 8 строк - относительно небольшое количество, которым можно пренебречь

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   age                   293 non-null    int64  
 1   rectal_temp           240 non-null    float64
 2   pulse                 274 non-null    float64
 3   temp_of_extremities   243 non-null    float64
 4   peristalsis           256 non-null    float64
 5   abdominal_distension  293 non-null    float64
 6   packed_cell_volume    271 non-null    float64
 7   total_protein         267 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 20.6 KB


In [19]:
for col in horse_filtered.columns:
    pct_missing = horse_filtered[col].isnull().mean()
    print(f'{col} - {pct_missing :.2%}')

age - 0.00%
rectal_temp - 18.09%
pulse - 6.48%
temp_of_extremities - 17.06%
peristalsis - 12.63%
abdominal_distension - 0.00%
packed_cell_volume - 7.51%
total_protein - 8.87%


Во время изучения данных мы столкнулись с тем, что для молодых и взрослых лошадей нормальные физиологические показатели различаются. Для температуры и пульса воспользуемся заполнением медианными значениями для групп молодые и взрослые лошади

In [20]:
fill_median_by_age = horse_filtered.copy()
fill_median_by_age.rectal_temp.fillna(horse_filtered.groupby('age').rectal_temp.transform('median'), inplace=True)

# видим, что меры центральной тенденции изменились незначительно
print(horse_filtered.rectal_temp.mean(), '->', fill_median_by_age.rectal_temp.mean())
print(horse_filtered.rectal_temp.median(), '->', fill_median_by_age.rectal_temp.median())
print(horse_filtered.rectal_temp.mode()[0], '->', fill_median_by_age.rectal_temp.mode()[0])
print(horse_filtered.rectal_temp.std(), '->',fill_median_by_age.rectal_temp.std())


38.16791666666669 -> 38.16245733788401
38.2 -> 38.1
38.0 -> 38.1
0.7322886641121578 -> 0.6649931267456192


In [21]:
fill_median_by_age.pulse.fillna(horse_filtered.groupby('age').pulse.transform('median'), inplace=True)

print(horse_filtered.pulse.mean(), '->', fill_median_by_age.pulse.mean())
print(horse_filtered.pulse.median(), '->', fill_median_by_age.pulse.median())
print(horse_filtered.pulse.mode()[0], '->', fill_median_by_age.pulse.mode()[0])
print(horse_filtered.pulse.std(), '->',fill_median_by_age.pulse.std())


71.95255474452554 -> 71.61433447098976
64.0 -> 60.0
48.0 -> 60.0
28.73118962357173 -> 28.258191914161962


In [22]:
# Заполнение по медиане дало бОльшие, чем ожидалось, изменения метрик. Попробуем заполнение средним.
fill_mean_by_age = horse_filtered.copy()
fill_mean_by_age.pulse.fillna(horse_filtered.groupby('age').pulse.transform('mean'), inplace=True)

print(horse_filtered.pulse.mean(), '->', fill_mean_by_age.pulse.mean())
print(horse_filtered.pulse.median(), '->', fill_mean_by_age.pulse.median())
print(horse_filtered.pulse.mode()[0], '->', fill_mean_by_age.pulse.mode()[0])
print(horse_filtered.pulse.std(), '->',fill_mean_by_age.pulse.std())
# Такой способ дает более близкие к исходным метрикам значения. примем его как рабочий вариант

71.95255474452554 -> 72.04307211559772
64.0 -> 66.0
48.0 -> 48.0
28.73118962357173 -> 28.125369932099026


In [23]:
# Применим то же к нашему датасету
horse_filtered.rectal_temp.fillna(horse_filtered.groupby('age').rectal_temp.transform('median'), inplace=True)
horse_filtered.pulse.fillna(horse_filtered.groupby('age').pulse.transform('mean'), inplace=True)


Температура конечностей - категориальный показатель. Мы знаем, что его мода = 3, однако теплые конечности должны коррелировать с повышенной температурой

In [24]:
print(horse_filtered.groupby('age').rectal_temp.median())

age
1    38.1
2    38.6
Name: rectal_temp, dtype: float64


Будем считать, что температура 39C для обоих возрастных групп является повышенной. И если показатель rectal_temp превышает это значение, будем указывать temp_of_extremities = 2 (теплые конечности), а если нет, то возьмем значение 3 по моде.

In [25]:
horse_filtered.loc[horse_filtered['rectal_temp'] > 39, 'temp_of_extremities'] = 2
horse_filtered.loc[horse_filtered.temp_of_extremities.isna(), 'temp_of_extremities'] = 3

Перистальтика и вздутие живота - взаимосвязанные параметры. При наличии вздутия перистальтика будет пониженной (из описания), соответственно, при нормальном состоянии живота скорее всего перистальтика будет нормальной. 
Заполним значения перистальтики следующим образом: если степерь вздутия живота проставлена, и это значение 1 (нет вздутия), перистальтику заполним значением 2 (норма); в остальных случаем заполним по моде значением 3

In [26]:
horse_filtered.loc[horse_filtered['abdominal_distension'] == 1, 'peristalsis'] = 2
horse_filtered.loc[horse_filtered.peristalsis.isna(), 'peristalsis'] = 3

Аналогично, для пропущенных значений столбца abdominal_distension, если в строке значение перистальтики заполнено и равно 3 или 4 (пониженная или отсутствует), для abdominal_distension укажем значение 3 (умеренная), в остальных случаях заполним по моде = 1

In [27]:
horse_filtered.loc[horse_filtered['peristalsis'].isin([3, 4]), 'abdominal_distension'] = 3
horse_filtered.loc[horse_filtered.abdominal_distension.isna(), 'abdominal_distension'] = 1

Остались числовые столбцы packed_cell_volume и total_protein. В случае с packed_cell_volume попробую заполнение по медиане, т.к. имеются выбросы сверху

In [29]:
# packed_cell_volume и заполнение по медиане
fill_median_by_age.packed_cell_volume.fillna(horse_filtered.groupby('age').packed_cell_volume.transform('median'), inplace=True)

print(horse_filtered.packed_cell_volume.mean(), '->', fill_median_by_age.packed_cell_volume.mean())
print(horse_filtered.packed_cell_volume.median(), '->', fill_median_by_age.packed_cell_volume.median())
print(horse_filtered.packed_cell_volume.mode()[0], '->', fill_median_by_age.packed_cell_volume.mode()[0])
print(horse_filtered.packed_cell_volume.std(), '->',fill_median_by_age.packed_cell_volume.std())

# ожидаемо изменилась мода, но другие метрики поменялись незначительно

46.29520295202952 -> 46.177474402730375
45.0 -> 45.0
37.0 -> 45.0
10.419334549704114 -> 10.033551818407293


В случае с total_protein более устойчивое к выбросам заполнение по медиане тоже кажется более приемлемым (несмотря на то, что математически выбросов мы не получили), т.к. диапазон данных широк (до 89 г/дЛ при норме 6 - 7.5) 

In [30]:
# total_protein и заполнение по медиане
fill_median_by_age.total_protein.fillna(horse_filtered.groupby('age').total_protein.transform('median'), inplace=True)

print(horse_filtered.total_protein.mean(), '->', fill_median_by_age.total_protein.mean())
print(horse_filtered.total_protein.median(), '->', fill_median_by_age.total_protein.median())
print(horse_filtered.total_protein.mode()[0], '->', fill_median_by_age.total_protein.mode()[0])
print(horse_filtered.total_protein.std(), '->',fill_median_by_age.total_protein.std())

24.456928838951317 -> 22.94368600682595
7.5 -> 7.5
6.5 -> 7.5
27.475009470785064 -> 26.669591466116703


In [34]:
# применим последние операции к нашему датасету
horse_filtered.packed_cell_volume.fillna(horse_filtered.groupby('age').packed_cell_volume.transform('median'), inplace=True)
horse_filtered.total_protein.fillna(horse_filtered.groupby('age').total_protein.transform('mean'), inplace=True)

In [35]:
# пропущенные данные убраны
for col in horse_filtered.columns:
    pct_missing = horse_filtered[col].isnull().mean()
    print(f'{col} - {pct_missing :.2%}')

age - 0.00%
rectal_temp - 0.00%
pulse - 0.00%
temp_of_extremities - 0.00%
peristalsis - 0.00%
abdominal_distension - 0.00%
packed_cell_volume - 0.00%
total_protein - 0.00%


## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам
Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.